## import

In [1]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

## setting

In [19]:
import os
from dotenv import load_dotenv
# .envファイルを読み込む
load_dotenv()

INPUT_VIDEO_PATH = os.getenv("INPUT_VIDEO_PATH", "default_input.mp4")
OUTPUT_VIDEO_PATH = os.getenv("OUTPUT_VIDEO_PATH", "processed_output.mp4")  # 音声なし映像

## exec

In [ ]:
# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOv8の顔検出モデル（事前に適切な顔検出モデルをダウンロード）
model = YOLO("yolov8n-face.pt").to(device)  # 顔検出用のYOLOモデル
model.half()  # FP16モードで動作（M3 Macはこの方が早い？）

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
output_video = OUTPUT_VIDEO_PATH

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

def apply_mosaic(image, x1, y1, x2, y2, mosaic_scale=0.1):
    """指定した座標の範囲にモザイクを適用"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # 縮小して拡大することでモザイク効果を作成
    small = cv2.resize(face, (max(1, int(w * mosaic_scale)), max(1, int(h * mosaic_scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    image[y1:y2, x1:x2] = mosaic
    return image

# フレーム処理
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 顔検出
    results = model(frame)

    # 各検出結果に対して処理
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
            frame = apply_mosaic(frame, x1, y1, x2, y2, mosaic_scale=0.05)

    # 出力動画に書き込み
    out.write(frame)

# 後処理
cap.release()
out.release()
print("動画処理が完了しました。output.mp4 に保存されました。")

ログを出力するから遅い可能性はないか？

In [12]:
# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOv8の顔検出モデル（事前に適切な顔検出モデルをダウンロード）
model = YOLO("yolov8n-face.pt").to(device)  # 顔検出用のYOLOモデル
model.half()  # FP16モードで動作（M3 Macはこの方が早い？）

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
output_video = OUTPUT_VIDEO_PATH

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

def apply_mosaic(image, x1, y1, x2, y2, mosaic_scale=0.1):
    """指定した座標の範囲にモザイクを適用"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # 縮小して拡大することでモザイク効果を作成
    small = cv2.resize(face, (max(1, int(w * mosaic_scale)), max(1, int(h * mosaic_scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    image[y1:y2, x1:x2] = mosaic
    return image

# フレーム処理
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 顔検出
    #results = model(frame)
    # ここでログを出力停止
    results = model(frame, stream=True, verbose=False)

    # 各検出結果に対して処理
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
            frame = apply_mosaic(frame, x1, y1, x2, y2, mosaic_scale=0.05)

    # 出力動画に書き込み
    out.write(frame)

# 後処理
cap.release()
out.release()
print("動画処理が完了しました。output.mp4 に保存されました。")

動画処理が完了しました。output.mp4 に保存されました。


02:28秒から02:06秒になったので少し早くなった。

In [14]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOv8の顔検出モデルをロード
model = YOLO("yolov8n-face.pt").to(device)
model.half()  # FP16モードで動作（ここで挿入！）

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
output_video = OUTPUT_VIDEO_PATH

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# バッチ処理設定（例: 5フレームずつまとめて処理）
batch_size = 5
frames = []

def apply_mosaic(image, x1, y1, x2, y2, mosaic_scale=0.1):
    """指定した座標の範囲にモザイクを適用"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # 縮小して拡大することでモザイク効果を作成
    small = cv2.resize(face, (max(1, int(w * mosaic_scale)), max(1, int(h * mosaic_scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    image[y1:y2, x1:x2] = mosaic
    return image

# フレーム処理
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frames.append(frame)

    # バッチサイズに達したら処理
    if len(frames) == batch_size:
        results = model(frames, stream=True, verbose=False)  # バッチ推論
        for i, result in enumerate(results):
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
                frames[i] = apply_mosaic(frames[i], x1, y1, x2, y2, mosaic_scale=0.05)
            out.write(frames[i])
        frames = []

# 残りのフレーム処理
for frame in frames:
    out.write(frame)

# 後処理
cap.release()
out.release()
print("動画処理が完了しました。output.mp4 に保存されました。")

動画処理が完了しました。output.mp4 に保存されました。


バッチでまとめて処理にすることで
01:49秒まで短縮

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOv8の顔検出モデルをロード
model = YOLO("yolov8n-face.pt").to(device)
model.half()  # FP16モードで動作（ここで挿入！）

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
output_video = OUTPUT_VIDEO_PATH

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# バッチ処理設定（例: 5フレームずつまとめて処理）
batch_size = 5
frames = []

def apply_mosaic(image, x1, y1, x2, y2, mosaic_scale=0.1):
    """指定した座標の範囲にモザイクを適用"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # 縮小して拡大することでモザイク効果を作成
    small = cv2.resize(face, (max(1, int(w * mosaic_scale)), max(1, int(h * mosaic_scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    image[y1:y2, x1:x2] = mosaic
    return image

# フレーム処理
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frames.append(frame)

    # バッチサイズに達したら処理
    if len(frames) == batch_size:
        results = model(frames, stream=True, verbose=False)  # バッチ推論
        for i, result in enumerate(results):
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
                frames[i] = apply_mosaic(frames[i], x1, y1, x2, y2, mosaic_scale=0.05)
            out.write(frames[i])
        frames = []

# 残りのフレーム処理
for frame in frames:
    out.write(frame)

# 後処理
cap.release()
out.release()
print("動画処理が完了しました。output.mp4 に保存されました。")

さらに高速化を試す

In [15]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import subprocess

# OpenCVのマルチスレッド無効化（シングルスレッドの方が速いことが多い）
cv2.setNumThreads(0)

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOモデルの読み込み（FP16でメモリ最適化）
model = YOLO("yolov8n-face.pt").to(device).half()

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
processed_video = OUTPUT_VIDEO_PATH  # 音声なし映像
output_video = "output.mp4"  # 最終的な音声付き動画

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定（音声なし）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(processed_video, fourcc, fps, (width, height))

# YOLOのバッチ処理を拡張（10フレームずつ処理）
batch_size = 10
frames = []

def fast_mosaic(image, x1, y1, x2, y2, scale=0.05):
    """NumPyベースの高速モザイク処理"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # NumPy + OpenCVを使ったモザイク処理
    small = cv2.resize(face, (max(1, int(w * scale)), max(1, int(h * scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    # 高速コピー
    np.copyto(image[y1:y2, x1:x2], mosaic)

# フレーム処理ループ
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frames.append(frame)

    # バッチ処理を実行
    if len(frames) == batch_size:
        results = model(frames, stream=True, verbose=False)  # バッチ推論
        for i, result in enumerate(results):
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
                fast_mosaic(frames[i], x1, y1, x2, y2, scale=0.05)
            out.write(frames[i])
        frames = []

# 残りのフレームを処理
for frame in frames:
    out.write(frame)

# 後処理
cap.release()
out.release()
print(f"映像処理が完了しました: {processed_video}")

映像処理が完了しました: /Users/takashi/Downloads/撮影テスト/output.mp4


さらなる高速化

In [16]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import subprocess

# OpenCVのマルチスレッド無効化（シングルスレッドの方が速いことが多い）
cv2.setNumThreads(0)

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOモデルの読み込み（FP16でメモリ最適化）
model = YOLO("yolov8n-face.pt").to(device).half()

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
processed_video = OUTPUT_VIDEO_PATH  # 音声なし映像
output_video = "output.mp4"  # 最終的な音声付き動画

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定（音声なし）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(processed_video, fourcc, fps, (width, height))

# YOLOのバッチ処理を拡張（10フレームずつ処理）
batch_size = 10
frames = []

# 直近の顔座標を保存する変数
previous_faces = []
frame_count = 0
detection_interval = 5  # 5フレームごとにYOLOで検出

def fast_mosaic(image, x1, y1, x2, y2, scale=0.05):
    """NumPyベースの高速モザイク処理"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # モザイク処理
    small = cv2.resize(face, (max(1, int(w * scale)), max(1, int(h * scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    # 高速コピー
    np.copyto(image[y1:y2, x1:x2], mosaic)

# フレーム処理ループ
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frames.append(frame)

    # 5フレームごとにYOLOで顔を検出
    if frame_count % detection_interval == 0:
        results = model(frames, stream=True, verbose=False)  # バッチ推論
        previous_faces = []  # 前のフレームの顔情報をクリア
        for i, result in enumerate(results):
            faces = []
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
                faces.append((x1, y1, x2, y2))
                fast_mosaic(frames[i], x1, y1, x2, y2, scale=0.05)
            previous_faces.append(faces)
            out.write(frames[i])
    else:
        # 前のフレームの顔座標をそのまま使用（YOLOの推論をスキップ）
        for faces in previous_faces:
            for x1, y1, x2, y2 in faces:
                fast_mosaic(frames[0], x1, y1, x2, y2, scale=0.05)
        out.write(frames[0])

    frames = []
    frame_count += 1

# 後処理
cap.release()
out.release()
print(f"映像処理が完了しました: {processed_video}")

映像処理が完了しました: /Users/takashi/Downloads/撮影テスト/output.mp4


シングルスレッドで動かして速度をチェック

In [17]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import subprocess

# OpenCVのマルチスレッド無効化（シングルスレッドで動作）
cv2.setNumThreads(1)

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOモデルの読み込み（FP16でメモリ最適化）
model = YOLO("yolov8n-face.pt").to(device).half()

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
processed_video = OUTPUT_VIDEO_PATH  # 音声なし映像
output_video = "output.mp4"  # 最終的な音声付き動画

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定（音声なし）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(processed_video, fourcc, fps, (width, height))

# 直近の顔座標を保存する変数
previous_faces = []
frame_count = 0
detection_interval = 5  # 5フレームごとにYOLOで検出

def fast_mosaic(image, x1, y1, x2, y2, scale=0.05):
    """NumPyベースの高速モザイク処理"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # モザイク処理
    small = cv2.resize(face, (max(1, int(w * scale)), max(1, int(h * scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    # 高速コピー
    np.copyto(image[y1:y2, x1:x2], mosaic)

# フレーム処理ループ（シングルスレッド版）
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 5フレームごとにYOLOで顔を検出
    if frame_count % detection_interval == 0:
        results = model(frame, verbose=False)  # シングルスレッドで1フレームずつ推論
        previous_faces = []  # 前のフレームの顔情報をクリア
        for result in results:
            faces = []
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
                faces.append((x1, y1, x2, y2))
                fast_mosaic(frame, x1, y1, x2, y2, scale=0.05)
            previous_faces.append(faces)
    else:
        # 前のフレームの顔座標をそのまま使用（YOLOの推論をスキップ）
        for faces in previous_faces:
            for x1, y1, x2, y2 in faces:
                fast_mosaic(frame, x1, y1, x2, y2, scale=0.05)

    out.write(frame)
    frame_count += 1

# 後処理
cap.release()
out.release()
print(f"映像処理が完了しました: {processed_video}")

映像処理が完了しました: /Users/takashi/Downloads/撮影テスト/output.mp4


シングルスレッド版にしたら01:31秒かかった

さらに10フレーム飛ばしにしてみる

In [20]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import subprocess

# OpenCVのマルチスレッド無効化（シングルスレッドで動作）
cv2.setNumThreads(1)

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOモデルの読み込み（FP16でメモリ最適化）
model = YOLO("yolov8n-face.pt").to(device).half()

# 入出力ファイル設定
input_video = INPUT_VIDEO_PATH
processed_video = OUTPUT_VIDEO_PATH  # 音声なし映像
output_video = "output.mp4"  # 最終的な音声付き動画

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定（音声なし）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(processed_video, fourcc, fps, (width, height))

# 直近の顔座標を保存する変数
previous_faces = []
frame_count = 0
detection_interval = 10  # 10フレームごとにYOLOで検出

def fast_mosaic(image, x1, y1, x2, y2, scale=0.05):
    """NumPyベースの高速モザイク処理"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # モザイク処理
    small = cv2.resize(face, (max(1, int(w * scale)), max(1, int(h * scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    # 高速コピー
    np.copyto(image[y1:y2, x1:x2], mosaic)

# フレーム処理ループ（シングルスレッド版）
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 5フレームごとにYOLOで顔を検出
    if frame_count % detection_interval == 0:
        results = model(frame, verbose=False)  # シングルスレッドで1フレームずつ推論
        previous_faces = []  # 前のフレームの顔情報をクリア
        for result in results:
            faces = []
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
                faces.append((x1, y1, x2, y2))
                fast_mosaic(frame, x1, y1, x2, y2, scale=0.05)
            previous_faces.append(faces)
    else:
        # 前のフレームの顔座標をそのまま使用（YOLOの推論をスキップ）
        for faces in previous_faces:
            for x1, y1, x2, y2 in faces:
                fast_mosaic(frame, x1, y1, x2, y2, scale=0.05)

    out.write(frame)
    frame_count += 1

# 後処理
cap.release()
out.release()
print(f"映像処理が完了しました: {processed_video}")

映像処理が完了しました: /Users/takashi/Downloads/撮影テスト/output.mp4


60フレーム同じ場所にモザイクを置き続けるようにしたら、01:17秒。

そこまで早くなってないし、モザイクがさすがにずれていく。

これではダメ。

マルチスレッドにして、もう少し長い動画で試すか。

あと音声を残したい・・・